In [1]:
import sqlite3
import pandas as pd

In [ ]:
#In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. We are going to find the answers to questions like:

#What is the total amount of debt that is owed by the countries listed in the dataset?
#Which country owns the maximum amount of debt and what does that amount look like?
#What is the average amount of debt owed by countries across different debt indicators?


In [2]:
conn = sqlite3.connect("pvworlddebt.db")

In [4]:
QUERY = """
SELECT name as Table_Name FROM
sqlite_master WHERE
type = 'table'
"""
df = pd.read_sql_query(QUERY, conn)
df

,Table_Name
0,international_debt


In [5]:
QUERY = """
SELECT *
FROM international_debt
"""
df = pd.read_sql_query(QUERY, conn)
df

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1
...,...,...,...,...,...
2352,Zimbabwe,ZWE,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,98492119.9
2353,Zimbabwe,ZWE,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,44396033.7
2354,Zimbabwe,ZWE,"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD,15761660.0
2355,Zimbabwe,ZWE,"Principal repayments on external debt, long-te...",DT.AMT.DLXF.CD,461632253.7


In [6]:
QUERY = """
SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries
FROM international_debt
"""
df = pd.read_sql_query(QUERY, conn)
df

,total_distinct_countries
0,124


In [7]:
#Finding out the distinct debt indicators
QUERY = """
SELECT DISTINCT(indicator_code) AS distinct_debt_indicators
FROM international_debt
ORDER BY 1
"""
df = pd.read_sql_query(QUERY, conn)
df

,distinct_debt_indicators
0,DT.AMT.BLAT.CD
1,DT.AMT.DLXF.CD
2,DT.AMT.DPNG.CD
3,DT.AMT.MLAT.CD
4,DT.AMT.OFFT.CD
5,DT.AMT.PBND.CD
6,DT.AMT.PCBK.CD
7,DT.AMT.PROP.CD
8,DT.AMT.PRVT.CD
9,DT.DIS.BLAT.CD


In [8]:
#Total amount of debt owed by all countries

QUERY = """
SELECT ROUND(SUM(debt)/1000000, 2) AS total_debt
FROM international_debt
"""
df = pd.read_sql_query(QUERY, conn)
df

,total_debt
0,3079734.49


In [11]:
#Country with the highest debt

QUERY = """
SELECT  country_name, SUM(debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1
"""
df = pd.read_sql_query(QUERY, conn)
df

,country_name,total_debt
0,China,2.857935e+11


In [12]:
#Now that we know that China has the highest debt in all of the world, we break down to see the average amount of debt across all indicators

QUERY = """
SELECT indicator_code AS debt_indicator, indicator_name, AVG(debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC
"""
df = pd.read_sql_query(QUERY, conn)
df

,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5.904868e+09
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5.161194e+09
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2.152041e+09
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1.958983e+09
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1.803694e+09
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1.644024e+09
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1.223139e+09
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1.220411e+09
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1.191188e+09
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1.082624e+09


In [14]:
#Finding out the highest amount of principal repayments
#It can be seen that DT.AMT.DFLX.CD tops the chart of average debtin the category of long term debts
#Which country owes the highest amount of debt 
QUERY = """
SELECT country_name, indicator_name
FROM international_debt
WHERE debt = (SELECT MAX(debt)
             FROM international_debt
             WHERE indicator_code LIKE 'DT.AMT.DLXF.CD')
"""
df = pd.read_sql_query(QUERY, conn)
df

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."


In [16]:
#Most Common Debt Indicator in which countries owe their debt?

QUERY = """
SELECT indicator_code, COUNT(indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC
LIMIT 20;
"""
df = pd.read_sql_query(QUERY, conn)
df

,indicator_code,indicator_count
0,DT.INT.OFFT.CD,124
1,DT.INT.MLAT.CD,124
2,DT.INT.DLXF.CD,124
3,DT.AMT.OFFT.CD,124
4,DT.AMT.MLAT.CD,124
5,DT.AMT.DLXF.CD,124
6,DT.DIS.DLXF.CD,123
7,DT.INT.BLAT.CD,122
8,DT.DIS.OFFT.CD,122
9,DT.AMT.BLAT.CD,122
